## Maaike notebook 1 for project 2: calculating correlation between mean income and housing energy labels in Amsterdam

In [ ]:
# Import packages
import numpy as np
import pandas as pd

In [ ]:
# Import income data file
income = pd.read_excel('2019_stadsdelen_3_15.xlsx', skiprows = [0,1,3,80,112,113])

income.head()

In [ ]:
income_df = income[['wijk/std', 'gemiddeld persoonlijk inkomen (x 1.000 euro)']]

In [ ]:
income_df = income_df.rename(columns = {'gemiddeld persoonlijk inkomen (x 1.000 euro)':'mean_income (x 1.000 euro)'})
income_df.head()

In [ ]:
income_df['area'] = income_df['wijk/std'].str.extract('([A-Z]\d\d)')
income_df['area_name'] = income_df['wijk/std'].str.replace('([A-Z]\d\d)', '')
income_df.head()

In [ ]:
income_df2 = income_df[['area', 'area_name', 'mean_income (x 1.000 euro)']]
income_df2.head()

In [ ]:
# Import energy label data
energy_labels = pd.read_csv('Energielabels_selectie gemeentes Amsterdam 4-1-2012.csv', sep = ';')

In [ ]:
energy_labels.head()

In [ ]:
list(energy_labels.columns)

In [ ]:
energy_labels_df = energy_labels[['Pand_postcode', 'PandVanMeting_energieklasse']]
energy_labels_df = energy_labels_df.rename(columns = {'PandVanMeting_energieklasse':'energy_class', 'Pand_postcode': 'postcode'})
energy_labels_df.head()

In [ ]:
# check energy_classes
labels_list = sorted(list(set(energy_labels_df['energy_class'])))
labels_list

In [ ]:
# add extra column with energy classes converted to numerical score
energy_labels_df['energy_class_score'] = energy_labels_df['energy_class'].replace({'A++': 9, 'A+': 8, 'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1})
energy_labels_df.head()

In [ ]:
# Import file with postcode to area conversion
postcode_to_area = pd.read_csv('PC6_VLAKKEN_BAG.csv', sep = ';')
postcode_to_area = postcode_to_area.rename(columns = {'Postcode6':'postcode'})
postcode_to_area.head()

In [ ]:
energy_labels_areas_df = pd.merge(energy_labels_df, 
                                  postcode_to_area[['postcode','Buurtcode']], 
                                  on = 'postcode', 
                                  how = 'left')

In [ ]:
energy_labels_areas_df.head()

In [ ]:
energy_labels_areas_df['area'] = energy_labels_areas_df['Buurtcode'].str.extract('([A-Z]\d\d)')

In [ ]:
energy_labels_areas_df.head()

In [ ]:
#create df with average energy class scores by area
energy_score_area = energy_labels_areas_df.groupby('area')[['energy_class_score']].mean()

energy_score_area.head()

In [ ]:
#merge income per area with energy class score per area 
energy_income_area = pd.merge(income_df2, energy_score_area, on = 'area', how = 'inner')
energy_income_area.head()

In [ ]:
#calculate correlation between mean energy class scores and mean income
corr_matrix = energy_income_area.corr()
corr_matrix
# There is no correlation between mean income and energy class score per area

In [ ]:
# Now: calculate the same per city district (stadsdeel)
# Import file with relationship city areas and districts

area_info = pd.read_csv('GEBIED_BUURTCOMBINATIES.csv', sep = ';')
area_info.head()


In [ ]:
#create df with area and district column
area_district = area_info[['Buurtcombinatie_code', 'Stadsdeel_code']]

#rename columns:
area_district = area_district.rename(columns = {'Buurtcombinatie_code': 'area','Stadsdeel_code': 'district'})
area_district.head()

In [ ]:
#add district info to energy income area df
energy_income_area_district = pd.merge(energy_income_area, area_district, on = 'area', how = 'inner')
energy_income_area_district.head()


In [ ]:
#create new df with mean income and energy score by district
energy_income_district = energy_income_area_district.groupby('district')[['mean_income (x 1.000 euro)','energy_class_score']].mean()

# Add district names to dataframe for interpretation purposes:

energy_income_district['district_names'] = ['Centrum', 'Westpoort', 'West', 'Nieuw-West', 'Zuid', 'Oost', 'Noord', 'Zuidoost']

energy_income_district.head(15)


In [ ]:
#calculate correlation between mean energy class scores and mean income per district
corr_matrix = energy_income_district.corr()
corr_matrix
# There is a negative correlation between mean income and energy class score per district

In [ ]:
# Export dataframe for future use
energy_income_district.to_csv('energy_income_district.csv', index=False)